In [25]:
 from itertools import product
from pathlib import Path

from PIL import Image, ImageChops


def summarise(img: Image.Image) -> Image.Image:
    """Summarise an image into a 16 x 16 image."""
    resized = img.resize((16, 16))
    return resized


def difference(img1: Image.Image, img2: Image.Image) -> float:
    """Find the difference between two images."""

    diff = ImageChops.difference(img1, img2)

    acc = 0
    width, height = diff.size
    for w, h in product(range(width), range(height)):
        #print(diff.getpixel((w, h)))
        r= diff.getpixel((w, h))[0]
        g= diff.getpixel((w, h))[1]
        b= diff.getpixel((w, h))[2]
        #print(r,g,b)
        acc += (r + g + b) / 3

    average_diff = acc / (width * height)
    normalised_diff = average_diff / 255
    return normalised_diff


def explore_directory(path: Path) -> None:
    """Find images in a directory and compare them all."""

    files = (
        #list(path.glob("*.jpg")) + list(path.glob("*.jpeg")) + list(path.glob("*.png"))
        list(path.glob("*.JPG"))
    )
    diffs = {}
    print(files)

    summaries = [(file, summarise(Image.open(file))) for file in files]

    for (f1, sum1), (f2, sum2) in product(summaries, repeat=2):
        key = tuple(sorted([str(f1), str(f2)]))
        if f1 == f2 or key in diffs:
            continue

        diff = difference(sum1, sum2)
        print(key, diff)
        diffs[key] = diff

    print()
    print("Near-duplicates found:")
    print("======================")
    for key, diff in diffs.items():
        if diff < 0.07:
            print(key)


if __name__ == "__main__":
    explore_directory(Path("/Users/rajeshwarrao/Desktop/XS10/1"))

[]

Near-duplicates found:


In [ ]:
import cv2
import os
import numpy as np

def find_similar_image(image_path, folder_path):
  """Finds the image in the given folder that has the most similar design to the given image.

  Args:
    image_path: The path to the image to be compared.
    folder_path: The path to the folder containing the images to be compared.

  Returns:
    The path to the image in the folder that has the most similar design to the given image.
  """

  # Read the image to be compared.
  image = cv2.imread(image_path)

  # Convert the image to grayscale.
  grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Find the features of the image to be compared.
  features = cv2.AKAZE(descriptor_size=16, descriptor_type=cv2.AKAZE_DESCRIPTOR_KAZE, threshold=0.001)

  features = features.detectAndCompute(grayscale_image, None)
  
  # Convert the features to a cv::UMat object.
  #features_cv = cv2.UMat(cv2.imcopy(features))

  # Find the best match in the folder.
  best_match = None
  best_match_score = 0.7

  for image_file in os.listdir(folder_path):
    if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
      # Read the image in the folder.
      image_in_folder = cv2.imread(os.path.join(folder_path, image_file))

      # Convert the image in the folder to grayscale.
      grayscale_image_in_folder = cv2.cvtColor(image_in_folder, cv2.COLOR_BGR2GRAY)

      # Find the features of the image in the folder.
      features2 = cv2.AKAZE(descriptor_size=16, descriptor_type=cv2.AKAZE_DESCRIPTOR_KAZE, threshold=0.001)

      features2 = features2.detectAndCompute(grayscale_image_in_folder, None)
      # Convert the features to a cv::UMat object.
      #features_in_folder_cv = cv2.UMat(np.ascontiguousarray(features_in_folder))

      # Calculate the similarity score between the two images.

      # Update the best match if the similarity score is better.
      score = np.sum(features[0] * features2[0])
      if similarity_score < best_match_score:
        best_match = image_file
        best_match_score = similarity_score

  return best_match

#result = find_similar_image("/Users/rajeshwarrao/Desktop/designpic1.png","/Users/rajeshwarrao/Desktop")
#print(result)

# Get the path to the image to be compared.
image_path = "/Users/rajeshwarrao/Desktop/designpic1.png"

# Get the path to the folder containing the images to be compared.
folder_path = "/Users/rajeshwarrao/Desktop"

# Find the image in the folder that has the most similar design to the given image.
best_match_path = find_similar_image(image_path, folder_path)

# Print the path to the best match.
print(best_match_path)